In [1]:
import pandas as pd
import sys

sys.path.append("/home/Kioedru/code/SSGO")
from codespace.utils.read_finetune_data import read_labels
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import torch

In [2]:
train_labels = read_labels("train", "P", "9606")
valid_labels = read_labels("valid", "P", "9606")
test_labels = read_labels("test", "P", "9606")
combine_labels = np.concatenate((train_labels, valid_labels), axis=0)
combine_labels = torch.from_numpy(combine_labels).float()
test_labels = torch.from_numpy(test_labels).float()

In [3]:
train_df = pd.read_pickle(
    "/home/Kioedru/code/SSGO/data/finetune/9606/train_data_P.pkl"
).dropna(subset=["Sequence"])
valid_df = pd.read_pickle(
    "/home/Kioedru/code/SSGO/data/finetune/9606/valid_data_P.pkl"
).dropna(subset=["Sequence"])
test_df = pd.read_pickle(
    "/home/Kioedru/code/SSGO/data/finetune/9606/test_data_P.pkl"
).dropna(subset=["Sequence"])

In [4]:
terms = pd.read_pickle("/home/Kioedru/code/SSGO/data/finetune/9606/terms_P.pkl")
mlb = MultiLabelBinarizer(classes=terms["terms"].tolist())

train_encode = np.array(mlb.fit_transform(train_df["annotations"]))
valid_encode = np.array(mlb.fit_transform(valid_df["annotations"]))
test_encode = np.array(mlb.fit_transform(test_df["annotations"]))
concat_encode = np.concatenate((train_encode, valid_encode))
display(concat_encode.shape, test_encode.shape)

(3460, 45)

(180, 45)

In [5]:
train_seq = train_df["Sequence"].tolist()
valid_seq = valid_df["Sequence"].tolist()
test_seq = test_df["Sequence"].tolist()
concat_seq = train_seq + valid_seq
display(len(concat_seq))

3460

划分数据集

In [6]:
train_sequences, test_sequences, train_labels, test_labels = (
    concat_seq,
    test_seq,
    concat_encode,
    test_encode,
)
train_labels = [np.array(item) for item in train_labels]
test_labels = [np.array(item) for item in test_labels]
display(train_labels[:5])

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]),
 array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])]

In [9]:
esm_path = "/home/Kioedru/code/SSGO/data/LLM/esm2"
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(esm_path, max_length=300, truncation=True)
# tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

我们的序列已经被转换成了 input_ids，这是标记化后的序列，还有一个 attention_mask。注意力掩码处理了长度可变的序列的情况——在这些情况下，较短的序列会用空白的“填充”标记进行填充，而注意力掩码会用0填充，表示模型应该忽略这些标记。

In [10]:
tokenizer(train_sequences[0])

{'input_ids': [0, 20, 6, 15, 10, 16, 21, 16, 15, 13, 15, 20, 19, 12, 11, 23, 5, 9, 19, 11, 21, 18, 19, 6, 6, 15, 15, 14, 13, 4, 14, 16, 11, 17, 18, 10, 10, 4, 14, 18, 13, 21, 23, 8, 4, 8, 4, 16, 14, 18, 7, 19, 14, 7, 23, 11, 14, 13, 6, 12, 7, 18, 13, 4, 4, 17, 12, 7, 14, 22, 4, 15, 15, 19, 6, 11, 17, 14, 8, 17, 6, 9, 15, 4, 13, 6, 10, 8, 4, 12, 15, 4, 17, 18, 8, 15, 17, 8, 9, 6, 15, 19, 21, 23, 14, 7, 4, 18, 11, 7, 18, 11, 17, 17, 11, 21, 12, 7, 5, 7, 10, 11, 11, 6, 17, 7, 19, 5, 19, 9, 5, 7, 9, 16, 4, 17, 12, 15, 5, 15, 17, 18, 10, 13, 4, 4, 11, 13, 9, 14, 18, 8, 10, 16, 13, 12, 12, 11, 4, 16, 13, 14, 11, 17, 4, 13, 15, 18, 17, 7, 8, 17, 18, 19, 21, 7, 15, 17, 17, 20, 15, 12, 12, 13, 14, 13, 9, 9, 15, 5, 15, 16, 13, 14, 8, 19, 19, 4, 15, 17, 11, 17, 5, 9, 11, 10, 9, 11, 4, 16, 9, 4, 19, 15, 9, 18, 15, 6, 13, 9, 12, 4, 5, 5, 11, 20, 15, 5, 14, 9, 15, 15, 15, 7, 13, 15, 4, 17, 5, 5, 21, 19, 8, 11, 6, 15, 7, 8, 5, 8, 18, 11, 8, 11, 5, 20, 7, 14, 9, 11, 11, 21, 9, 5, 5, 5, 12, 13, 9, 13, 

In [11]:
train_tokenized = tokenizer(train_sequences)
test_tokenized = tokenizer(test_sequences)

In [ ]:
display(len(train_tokenized["input_ids"]))

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)

train_dataset

In [ ]:
train_labels = [label.astype(np.float32) for label in train_labels]
test_labels = [label.astype(np.float32) for label in test_labels]

In [ ]:
train_dataset = train_dataset.add_column("labels", train_labels)
test_dataset = test_dataset.add_column("labels", test_labels)
train_dataset

In [6]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EsmForSequenceClassification,
)

num_labels = 45  # Add 1 since 0 can be a label
# model = EsmForSequenceClassification.from_pretrained(
#     esm_path, num_labels=num_labels, problem_type="multi_label_classification"
# )
model = EsmForSequenceClassification.from_pretrained(
    esm_path,
    num_labels=num_labels,
    problem_type="multi_label_classification",
).to("cuda:1")
print(model)

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at /home/Kioedru/code/SSGO/data/LLM/esm2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForSequenceClassification(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-11): 12 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-0

这些警告告诉我们模型正在丢弃用于语言建模的一些权重（lm_head），并添加用于序列分类的一些权重（classifier）。这正是我们在想要在序列分类任务上微调语言模型时所期望的！


接下来，我们初始化我们的 TrainingArguments。这些参数控制各种训练超参数，并将被传递给我们的 Trainer。


In [ ]:
# 设置超参
batch_size = 1

args = TrainingArguments(
    output_dir="/home/Kioedru/code/SSGO/codespace/result",
    evaluation_strategy="epoch",  # 指定评估策略为每个epoch结束后进行评估
    save_strategy="epoch",  # 设置为每个epoch结束后保存模型
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,  # 每个设备上的训练批次大小
    per_device_eval_batch_size=batch_size,  # 每个设备上的评估批次大小
    num_train_epochs=3,  # 训练的总epoch数量
    weight_decay=0.01,  # 权重衰减（L2正则化）的强度，有助于防止过拟合
    load_best_model_at_end=True,  # 在评估指标（这里是准确率）最佳时保存并加载该模型
    metric_for_best_model="accuracy",  # 选择最佳模型的评估指标
    push_to_hub=False,  # 是否将模型推送到hf
)

定义用于评估模型的指标

In [ ]:
from evaluate import load
import numpy as np

metric = load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    labels = labels.reshape((-1,))
    predictions = np.argmax(predictions, axis=2)
    predictions = predictions.reshape((-1,))
    predictions = predictions[labels != -100]
    labels = labels[labels != -100]
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,  # 此时传递tokenizer用于填充序列
    # compute_metrics=compute_metrics,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
import sys

sys.path.append("/home/Kioedru/code/SSGO")
from codespace.model import aslloss_adaptive

loss = aslloss_adaptive.AsymmetricLossOptimized(
    gamma_neg=int(2),
    gamma_pos=int(0),
    clip=float(0),
    disable_torch_grad_focal_loss=1e-5,
    eps=args.eps,
)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, model):
        super(Model, self).__init__()
        self.pretrained = EsmModel.from_pretrained(
            "/home/Kioedru/code/SSGO/data/LLM/esm2"
        )
        self.fc = torch.nn.Linear(in_features=480, out_features=45)

    def forward(self, input_ids, attention_mask, labels=None):
        with torch.no_grad():
            last_hidden_state = self.pretrained(
                input_ids=input_ids, attention_mask=attention_mask
            ).last_hidden_state

        output = self.fc(last_hidden_state)

        loss = None
        if labels is not None:
            loss = torch.nn.functional.cross_entropy(output, labels)

        return loss, output